<a href="https://colab.research.google.com/github/SBhat2615/MIT-academics/blob/main/SEM6/PCAP/LAB/working_cuda_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# To show that if there is cuda tookit installed
!ls /usr/local

bin    cuda	cuda-11.8  games	       include	lib64	   man	 share
colab  cuda-11	etc	   _gcs_config_ops.so  lib	licensing  sbin  src


In [ ]:
# To show that if we have the nvcc command
!which nvcc

/usr/local/cuda/bin/nvcc


In [ ]:
# To show the property of the nvidia card(On my one, I use the K80)
!nvidia-smi

Wed Apr 19 03:37:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%writefile final.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>
__global__ void rotate(char *s, int *index, char *res, int s1, int s2, int n)
{
	int blockid = blockIdx.x, tid = threadIdx.x, start = index[blockid], end = index[blockid+1], size = start-end-1, m = n*size;
	if(tid < size){
		res[start+tid] = s[start+(tid+m)%size]
	}

	
}
	
int main(void)
{
	int *d_s, *d_index;
	char s[20], res[20];
	int index[10];

	printf("Enter string s:\n");
	scanf("%[^\n]", s);

	int size1 = strlen(s)*sizeof(char);
	index[0] = 0;
	int j = 0;
	for(int i = 0; i < size1; i++){
		if(s[i] == ' '){
			if(maxi < i-index[j])
				maxi = i-index[j];
			index[++j] = i+1;
		}
	}

	int size2 = j*sizeof(int);

	cudaMalloc((void**)&d_s, size1);
	cudaMalloc((void**)&d_res, size1);
	cudaMalloc((void**)&d_index, size2);

	cudaMemcpy(d_s, s, size1, cudaMemcpyHostToDevice);
	cudaMemcpy(d_index, index, size2, cudaMemcpyHostToDevice);

	dim3 grid(j);
	dim3 block(maxi);
	rotate<<<grid, block>>>(d_s, d_index, d_res, size1, size2, 3);

	cudaMemcpy(res, d_res, size1, cudaMemcpyDeviceToHost);
	//printf("cuda successfully ran!\n");

	printf("Result string is:\n");
	printf("%s", res);
	getchar();
	cudaFree(d_a);
	cudaFree(d_b);
	cudaFree(d_t);
	return 0;
}

Overwriting l9q1a.cu


In [ ]:
!nvcc final.cu -o final

nvcc warning : The 'compute_35', 'compute_37', 'sm_35', and 'sm_37' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [ ]:
!./final

Enter the value of m: 2
Enter the value of n: 2
Enter input matrix A:
2 4 5 6
Enter input matrix B:
7 8 3 2
cuda successfully ran!
cuda successfully ran!
Result matrix is:
9	12	
8	8	


In [14]:
%%writefile example.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>
__global__ void operate(int* a, int* b, int n, int m)
{
  printf("Kernel called !!\n");
  int cid = threadIdx.y * blockDim.x + threadIdx.x;
  int rid = threadIdx.x * blockDim.y + threadIdx.y;
  printf("rid = %d && cid = %d\n", rid, cid);
  if(cid != 0 || rid != 0 || cid != n || rid != m){
    b[rid*m+cid] = 0;
    printf("first - last\n");
  }
  else{
    b[rid*m+cid] = 1;
    printf("middle");
  }
}
	
int main(void)
{
	int m, n, size;
  int *a, *b;
  int *d_a, *d_b;

  printf("Enter size of matrix: ");
  scanf("%d %d", &m, &n);

  size = sizeof(int)*m*n;
  a = (int*)malloc(size);
  b = (int*)malloc(size);

	printf("Enter matrix A:\n");
	for(int i = 0; i < m*n; i++){
    scanf("%d", &a[i]);
  }

	cudaMalloc((void**)&d_a, size);
  cudaMalloc((void**)&d_b, size);

	cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);

	operate<<<1, (m,n)>>>(d_a, d_b, n, m);

  cudaMemcpy(b, d_b, size, cudaMemcpyDeviceToHost);

	printf("Matrix result:\n");
  for(int i = 0; i < m; i++){
    for(int j = 0; j < n; j++){
      printf("%d ", b[i*n + j]);
    }
    printf("\n");
  }
  
	getchar();
	cudaFree(d_a);
	return 0;
}

Overwriting example.cu


In [15]:
!nvcc example.cu

example.cu(44): warning #174-D: expression has no effect

example.cu(44): warning #174-D: expression has no effect



In [16]:
!./a.out

Enter size of matrix: 4 4
Enter matrix A:
1 2 3 4 6 5 8 3 2 4 10 1 9 1 2 5
Kernel called !!
Kernel called !!
Kernel called !!
Kernel called !!
rid = 0 && cid = 0rid = 1 && cid = 1rid = 2 && cid = 2rid = 3 && cid = 3first - last
first - last
first - last
first - last
Matrix result:
0 0 0 0 
0 0 0 0 
0 0 0 0 
0 0 0 0 
